<a href="https://colab.research.google.com/github/IssamMerikhi/CCD/blob/master/jupyter/func_GC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
def process():
  import pandas as pd
  import numpy as np
  import matplotlib.pyplot as plt
  from sklearn.preprocessing import MinMaxScaler


  gdp = pd.read_csv("GDP.csv", index_col="DATE", parse_dates=True)
  money = pd.read_csv("MoneyM3.csv", index_col="DATE", parse_dates=True)


  gdp = gdp.rename(columns = {'GDP':'gdp'}, inplace = False)
  money = money.rename(columns = {'M3':'money'}, inplace = False)
  
  df = pd.merge(gdp, money, left_index=True, right_index=True)
  from sklearn.preprocessing import MinMaxScaler

  scaler = MinMaxScaler()

  df[['gdp', 'money']] = scaler.fit_transform(df[['gdp', 'money']])
  

  avgs = df.mean()
  devs = df.std()

  for col in df.columns:
    df[col] = (df[col] - avgs[col]) / devs.loc[col]

  df = df.diff().dropna()

  df.reset_index(level=0, inplace=True)
  del df['DATE']

  return df

In [10]:
def grangercausality_significance(df, lvl):
  from statsmodels.tsa.stattools import grangercausalitytests
  import pandas as pd

  n = round(len(df)/4)
  gct = grangercausalitytests(df, n, verbose=False)

  Ftest = []
  Chi2test = []
  lrtest = []
  lag = []
  results = []

  for i in range(0, n):
    results.append(list(gct.values())[i])
    Ftest.append(results[i][0]['ssr_ftest'][1])
    Chi2test.append(results[i][0]['ssr_chi2test'][1])
    lrtest.append(results[i][0]['lrtest'][1])
    lag.append(i+1)


  d = {'Ftest':Ftest,'Chi2 test':Chi2test,'LR test': lrtest,'lags':lag}
  df2 = pd.DataFrame(d)

  df2.drop(df2[df2.Ftest > lvl].index, inplace=True)

  return df2

In [11]:
df_test = process()

grangercausality_significance(df_test, 0.1)

,Ftest,Chi2 test,LR test,lags
13,0.031483,1.083242e-03,4.870807e-03,14
14,0.011841,8.694127e-05,9.290136e-04,15
15,0.018083,1.105383e-04,1.247495e-03,16
16,0.025700,1.232501e-04,1.527684e-03,17
17,0.026665,6.396895e-05,1.140103e-03,18
18,0.021005,1.449669e-05,5.208505e-04,19
19,0.028661,1.318022e-05,5.751157e-04,20
20,0.037752,1.081985e-05,5.994032e-04,21
21,0.037801,3.472021e-06,3.677015e-04,22
22,0.062750,6.193167e-06,6.029481e-04,23
